## Keras model - Deeper2
---- 
* **Why Keras ? **


 > 실질적으로 풀려는 문제가 풀리는 문제 일까 아닐까를 정의가 안된 시점에서, 얕은 모델 부터 깊고 복잡한 모델까지 모두 실험해 보기 위해서 모델을 만들고 러닝 하여, 결과를 확인하는 과정에 사이클을 짧게 하기위함이 첫번째 이유이고, 두번째는 데이터를 직접 resizing하지 않아서 tfrecord로 파싱하는데 오류가 있었기 때문이다. 
 
* **현재 모델**
> **VGG alike** : 맨 처음 1x1conv layer로 gray scals 을 만들고 다음 VGG와 비슷한 구조로 conv을 수행한 후 FCN으로 넘어간다. 


> Deeper와 다른점 
 1.Dense layer 한 개 추가 
 2.1x1 제거 


In [3]:
import numpy as np 
import pickle 

# load dataset 
# with open('../sample_image/sample560_crop_df.bin', 'rb') as f :
#     crop_img = pickle.load(f)

with open('../sample_image/final_df.bin', 'rb') as f :
    crop_img = pickle.load(f)

    
X = np.array([i for i in crop_img.crop_img_05])
y = crop_img.weight.values

In [5]:
y = crop_img.weight.values

In [13]:
#Parameters 

BATCH_SIZE = 24

In [14]:
### test/train split 
idx = list(np.random.randint(0, len(X), int(len(X) * 0.8)))
test_idx = list(set(list(range(len(X)))) - set(idx))

X_train = X[idx]
y_train = y[idx]
X_test = X[test_idx]
y_test = y[test_idx]

In [15]:
# image augmentation 
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

gen.fit(X_train, augment=True)

Using TensorFlow backend.


In [33]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.layers import Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2


def cnn_model():
    model = Sequential()
    #  first layer for making images to the gray scale images, more precisely 
#     model.add(Conv2D(1, (1, 1), activation='relu', input_shape=(125, 255, 3), padding='same', kernel_regularizer=l2(0.001)))

    # 
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=(125, 275, 3), kernel_regularizer=l2(0.001)))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())

    model.add(Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())

    model.add(Conv2D(64, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())

    model.add(Flatten())
#     model.add(Dense(500, activation='relu', kernel_regularizer=l2(0.001)))
#     model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    return model 



In [34]:
from keras.utils.training_utils import multi_gpu_model

model = cnn_model()
model = multi_gpu_model(model, gpus=2)
model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['mae'])

In [35]:
%%time
hist = model.fit_generator(gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                           steps_per_epoch=len(X_train)/BATCH_SIZE,
                          epochs=200, validation_data=(X_test, y_test), verbose=2)

Epoch 1/200
 - 8s - loss: 2495.5730 - mean_absolute_error: 39.5988 - val_loss: 9336572482.0645 - val_mean_absolute_error: 96459.8180
Epoch 2/200
 - 4s - loss: 614.6873 - mean_absolute_error: 19.2886 - val_loss: 3534007254.7097 - val_mean_absolute_error: 59319.9268
Epoch 3/200
 - 4s - loss: 351.1324 - mean_absolute_error: 15.1692 - val_loss: 2519848200.2581 - val_mean_absolute_error: 50099.1764
Epoch 4/200
 - 4s - loss: 323.9229 - mean_absolute_error: 14.4131 - val_loss: 2106873913.8065 - val_mean_absolute_error: 45790.8300
Epoch 5/200
 - 4s - loss: 389.5905 - mean_absolute_error: 15.7238 - val_loss: 1004589685.6774 - val_mean_absolute_error: 31603.0554
Epoch 6/200
 - 4s - loss: 330.6545 - mean_absolute_error: 14.7842 - val_loss: 1788580029.9355 - val_mean_absolute_error: 42165.1155
Epoch 7/200
 - 4s - loss: 336.1355 - mean_absolute_error: 14.5824 - val_loss: 174701686.1935 - val_mean_absolute_error: 13135.1391
Epoch 8/200
 - 4s - loss: 390.5811 - mean_absolute_error: 15.5456 - val_loss

Epoch 63/200
 - 4s - loss: 277.0155 - mean_absolute_error: 13.5824 - val_loss: 236581622.7097 - val_mean_absolute_error: 15322.5679
Epoch 64/200
 - 4s - loss: 243.4490 - mean_absolute_error: 12.3706 - val_loss: 564085241.8065 - val_mean_absolute_error: 23488.5159
Epoch 65/200
 - 4s - loss: 256.5140 - mean_absolute_error: 13.0096 - val_loss: 861190670.4516 - val_mean_absolute_error: 29204.2445
Epoch 66/200
 - 4s - loss: 239.1160 - mean_absolute_error: 12.3225 - val_loss: 304737113.2903 - val_mean_absolute_error: 17407.6771
Epoch 67/200
 - 4s - loss: 249.9849 - mean_absolute_error: 12.4981 - val_loss: 494916176.5161 - val_mean_absolute_error: 22151.1930
Epoch 68/200
 - 4s - loss: 212.0340 - mean_absolute_error: 11.3627 - val_loss: 54641309.0323 - val_mean_absolute_error: 7360.0208
Epoch 69/200
 - 4s - loss: 233.5149 - mean_absolute_error: 12.1045 - val_loss: 67836490.8387 - val_mean_absolute_error: 8202.8584
Epoch 70/200
 - 4s - loss: 262.9038 - mean_absolute_error: 12.8819 - val_loss: 4

KeyboardInterrupt: 

In [36]:
import matplotlib.pyplot as plt
%matplotlib inline


plt.figure(figsize=(9,9))
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_mean_absolute_error"])
plt.show()

NameError: name 'hist' is not defined

<Figure size 648x648 with 0 Axes>

In [ ]:
r = 2
print("prediction : {}, Truth : {} ".format(model.predict(X_test)[r], y_test[r]))